# **Mini-project II.**

# "Deep Learning with Tensorflow and Keras". MNIST dataset

# Done by **Baglan AITU**.

### Import the relevant packages

In [ ]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfsd

In [ ]:
mnist_dataset,mnist_info=tfsd.load(name='mnist',with_info=True, as_supervised=True, shuffle_files=True)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


### Data

### Train and test datasets

In [43]:
mnist_train,mnist_test = mnist_dataset['train'],mnist_dataset['test']
num_validation_samples = 0.0001*mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples,tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples,tf.int64)



### Normalize train and test datasets

In [44]:
def scale(image,label):
    image=tf.cast(image,tf.float32)
    image/=255.
    return image,label

scaled_train_and_validation_data=mnist_train.map(scale)
scaled_test_data = mnist_test.map(scale)

### Shuffle Train dataset ang separate train and validation samples

In [45]:
BUFFER_SIZE=10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

### Prepare for batching

In [46]:
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE=30
train_data=train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = scaled_test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

### Model

In [47]:
input_size = 784
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation='tanh'),
    tf.keras.layers.Dense(hidden_layer_size,activation='sigmoid'),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(output_size,activation='softmax')
])

### Choose the optimizer and the loss function

In [48]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

### Training

In [49]:
NUM_EPOCHS=3

model.fit(train_data,epochs=NUM_EPOCHS,validation_data=(validation_inputs, validation_targets),verbose=2)

Epoch 1/3
2000/2000 - 5s - loss: 0.3366 - accuracy: 0.8985 - val_loss: 0.0613 - val_accuracy: 1.0000
Epoch 2/3
2000/2000 - 5s - loss: 0.1343 - accuracy: 0.9597 - val_loss: 0.0125 - val_accuracy: 1.0000
Epoch 3/3
2000/2000 - 5s - loss: 0.1062 - accuracy: 0.9682 - val_loss: 0.0125 - val_accuracy: 1.0000


### Test the model

In [50]:
test_loss,test_accuracy=model.evaluate(test_data)

1/1 [==============================] - 1s 561ms/step - loss: 0.1113 - accuracy: 0.9676
